# Prediction Pipeline: Tree-Based

In [ ]:
import itertools
import os
import pandas as pd
import parquet

from tree_predictor import TreePredictor
from utils import Utils

In [ ]:
config = Utils.read_config_for_env(config_path='../config/config.yml')
treepredictor = TreePredictor(config)

## Data Preparation

## Prepare monthly training and validation data

In [ ]:
def data_prep_pipeline(
        df_daily,
        splitname,
        refresh):
    
    # get base monthly data
    df_base = treepredictor.data.get_monthly_data(df_daily, splitname, refresh)

    # get monthly data with lag and ma features
    df_ts= treepredictor.data.get_ts_data(df_base, splitname, refresh, treepredictor.num_lag_mon)
    
    # TODO: scaling

    # TODO: OHE

    return df_ts

### Training data

In [ ]:
columns_needed = ['monthly_period', 'shop_id', 'item_id', 'item_category_id', 'amount', 'price']
df_daily_train = treepredictor.df_daily_train[columns_needed].copy()
# df_daily_train.info()
df_train = data_prep_pipeline(
    df_daily_train,
    'train',
    refresh=False)

# create X and y
train_y = df_train['amount_item']
df_train.drop(columns=['price', 'amount_item', 'amount_cat'], axis=1, inplace=True)
train_X = df_train


### Validation data

In [ ]:
df_daily_val = treepredictor.df_daily_val[columns_needed].copy()
# df_daily_train.info()
df_val = data_prep_pipeline(
    df_daily_val,
    'val',
    refresh=True)

# create X and y
val_y = df_val['amount_item']
df_val.drop(columns=['price', 'amount_item', 'amount_cat'], axis=1, inplace=True)
val_X = df_val